# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityData = pd.read_csv('City Data.csv')
cityData.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Kollam,40,IN,1587266785,62,8.88,76.60,89.60,4.70
1,Laas,74,IT,1587266427,87,46.62,10.70,48.00,3.36
2,Monrovia,100,LR,1587266786,83,6.30,-10.80,80.58,6.62
3,Tilichiki,32,RU,1587266786,92,60.47,166.10,12.70,12.01
4,Bantry,90,IE,1587266787,100,51.68,-9.45,46.40,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Get key
gmaps.configure(g_key)

# Store latitude and longitude in locations
locations = cityData[['Lat', 'Lng']]
humidity = cityData['Humidity'].astype(float)

# Plot Heatmap and set place
fig = gmaps.figure(zoom_level= 2.2,center = (14,10))


# Create heat layer
heatL = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 200,
                                 point_radius = 3)

#Add layer
fig.add_layer(heatL)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
citiesFitw = cityData.loc[(cityData['Max Temp'] > 65) &
                         (cityData['Max Temp'] < 80) &
                         (cityData['Cloudiness'] == 0) &
                         (cityData['Wind Speed'] < 10)]

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
citiesFitw = citiesFitw.dropna()
citiesFitw = citiesFitw.reset_index()
citiesFitw.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,119,Luján,0,AR,1587266810,76,-34.57,-59.10,66.00,1.99
1,218,Santa Cruz de la Sierra,0,BO,1587266205,68,-17.80,-63.17,66.20,5.82
2,343,Morondava,0,MG,1587266856,79,-20.28,44.28,74.55,7.58
3,355,Baharly,0,TM,1587266858,36,38.44,57.43,67.89,8.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = citiesFitw
hotel_df.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,119,Luján,0,AR,1587266810,76,-34.57,-59.10,66.00,1.99
1,218,Santa Cruz de la Sierra,0,BO,1587266205,68,-17.80,-63.17,66.20,5.82
2,343,Morondava,0,MG,1587266856,79,-20.28,44.28,74.55,7.58
3,355,Baharly,0,TM,1587266858,36,38.44,57.43,67.89,8.55


In [13]:
hotel = []

for index, row in hotel_df.iterrows():
    # Set parameters
    params = {'radius': 5000,
              'types': 'hotel',
              'keyword': 'hotel',
              'key': g_key,
              'location': f'{row["Lat"]}, {row["Lng"]}'}
              
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params).json()
    
    try:
        hotel.append(response['results'][0]['name'])
    except IndexError:
        hotel.append(np.nan)
        
hotel_df['Hotel Name'] = hotel
hotel_df.dropna()
hotel_df.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,119,Luján,0,AR,1587266810,76,-34.57,-59.10,66.00,1.99,Howard Johnson Resort Spa & Convention Center ...
1,218,Santa Cruz de la Sierra,0,BO,1587266205,68,-17.80,-63.17,66.20,5.82,Los Tajibos Hotel & Convention Center
2,343,Morondava,0,MG,1587266856,79,-20.28,44.28,74.55,7.58,Palissandre Cote Ouest resort & SPA
3,355,Baharly,0,TM,1587266858,36,38.44,57.43,67.89,8.55,NaN


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in citiesFitw.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer (locations, info_box_content = hotel_info)

# Add the layer to the map

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))